In [14]:
import requests
import json
from openai import AzureOpenAI
import logging

# Thêm logging configuration ở đầu file
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Thiết lập API key cho Azure OpenAI và Airtable
azure_api_key = ""  # Thay bằng API key của bạn
airtable_api_key = ""  # Thay bằng Airtable API key
base_id = "app7mcJMf9O8GQRal"  # Base ID của bạn
table_name = "tiktok_post"  # Table ID của bạn
view = "0_extract_2"  # View ID của bạn

azure_endpoint = "https://stepup.openai.azure.com"  # Thay bằng endpoint của bạn
api_version = "2024-08-01-preview"  # Phiên bản API
openai_model = "gpt-4o-mini"  # Model OpenAI

# Thiết lập client Azure OpenAI
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=api_version
)

# Prompt hệ thống
system_prompt = """
Bạn là chuyên gia marketing, thành thạo phân tích video và social media \
Nhiệm vụ của bạn là khai thác insights từ các comment của user về một video liên quan đến việc học hoặc giao tiếp tiếng Anh\
Đọc và tìm ra những thông tin sau, trả kết quả dạng JSON\n\n

1. AI_comments_persona: chân dung những user đang comment, có thể segmentation và đánh giá sơ bộ\n

2. AI_comments_sensitive: Đánh giá phản ứng chung của user khi comment và giải thích tại sao\n

3. AI_comments_painpoints: Liệt kê các pain points của user được nhắc đến, gom thành các nhóm:\n
- Động lực \n
- Tâm lý\n
- Chi phí\n
- Kiến thức chuyên môn: từ vựng, ngữ pháp, phát âm, phản xạ...\n
- Trải nghiệm sản phẩm\n
- Khác\n\n

4. AI_comments_needs: Nhu cầu, khao khát của người dùng trong việc học hoặc giao tiếp tiếng anh\n\n

5. AI_comments_hidden_insights: Bạn nghĩ là có insights gì ẩn sâu phía sau mà thâm chí user không biết?\n

"""

def fetch_airtable_data(base_id, table_name, view, api_key, offset=None):
    try:
        url = f"https://api.airtable.com/v0/{base_id}/{table_name}"
        headers = {
            "Authorization": f"Bearer {api_key}",
        }
        params = {
            "view": view
        }
        if offset:
            params["offset"] = offset
            
        logger.info(f"Fetching data from Airtable with params: {params}")
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            logger.info(f"Successfully fetched {len(data.get('records', []))} records")
            return data
        else:
            logger.error(f"Failed to fetch data. Status code: {response.status_code}")
            logger.error(f"Response: {response.text}")
            return None
    except Exception as e:
        logger.error(f"Error fetching data: {str(e)}")
        return None

def update_airtable_record(base_id, table_name, api_key, record_id, fields):
    url = f"https://api.airtable.com/v0/{base_id}/{table_name}/{record_id}"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {"fields": fields}
    response = requests.patch(url, headers=headers, json=data)
    if response.status_code != 200:
        logger.error(f"Failed to update record {record_id}. Status code: {response.status_code}, response: {response.text}")
    else:
        logger.info(f"Successfully updated record {record_id}")

def get_completion(prompt, record_id):
    try:
        logger.info(f"Sending request to OpenAI for record {record_id}")
        response = client.chat.completions.create(
            model=openai_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            top_p=1,
            presence_penalty=0,
            frequency_penalty=0,
            max_tokens=4000
        )
        logger.info(f"Received response from OpenAI for record {record_id}")
        return response
    except Exception as e:
        logger.error(f"Error in OpenAI API call for record {record_id}: {e}")
        return None

def test_openai_connection():
    try:
        logger.info("Kiểm tra kết nối Azure OpenAI")
        response = client.chat.completions.create(
            model=openai_model,
            messages=[{"role": "user", "content": "Hello"}],
            max_tokens=10
        )
        logger.info("Kết nối Azure OpenAI thành công")
        return True
    except Exception as e:
        logger.error(f"Lỗi kết nối Azure OpenAI: {str(e)}")
        return False

def format_reverse_to_bullets(data_dict):
    if not isinstance(data_dict, dict):
        return data_dict
        
    formatted_text = ""
    for key, value in data_dict.items():
        if isinstance(value, list):
            formatted_text += f"- {key}:\n"
            for item in value:
                formatted_text += f"  + {item}\n"
        else:
            formatted_text += f"- {key}: {value}\n"
    return formatted_text.strip()

def format_painpoints(painpoints_dict):
    if not isinstance(painpoints_dict, dict):
        return str(painpoints_dict)
        
    formatted_text = []
    for category, points in painpoints_dict.items():
        if isinstance(points, list):
            formatted_text.extend(points)
    return ", ".join(formatted_text)

def format_needs(needs_data):
    if isinstance(needs_data, list):
        return "\n".join(f"- {need}" for need in needs_data)
    elif isinstance(needs_data, dict):
        return format_reverse_to_bullets(needs_data)
    else:
        return str(needs_data)

def get_hidden_insights(insights_data):
    try:
        if isinstance(insights_data, dict):
            return str(insights_data.get("insight", insights_data.get("insights", "no information")))
        elif isinstance(insights_data, list):
            # Nếu là list, lấy phần tử đầu tiên và chuyển thành string
            return str(insights_data[0]) if insights_data else "no information"
        elif isinstance(insights_data, str):
            return insights_data
        else:
            return "no information"
    except Exception as e:
        logger.error(f"Lỗi xử lý hidden insights: {str(e)}")
        return "no information"

def main():
    try:
        logger.info("Bắt đầu chạy chương trình")
        offset = None
        while True:
            logger.info("Đang fetch dữ liệu từ Airtable...")
            data = fetch_airtable_data(base_id, table_name, view, airtable_api_key, offset)

            if not data:
                logger.warning("Không lấy được dữ liệu từ Airtable")
                break

            records = data.get("records", [])
            logger.info(f"Số lượng records lấy được: {len(records)}")

            for record in records:
                record_id = record.get("id")
                logger.info(f"Đang xử lý record ID: {record_id}")
                
                fields = record.get("fields", {})
                title = fields.get("title", "")
                content_text = fields.get("desc", "")
                transcript = fields.get("transcript", "")
                comments = fields.get("comments_1", "")

                logger.info(f"Dữ liệu của record {record_id}:")
                logger.info(f"Title: {title[:50]}...")
                logger.info(f"Content: {content_text[:50]}...")
                logger.info(f"Script: {transcript[:50]}...")
                logger.info(f"Comments: {comments[:50]}...")

                if content_text:  # Chỉ kiểm tra content_text
                    logger.info(f"Đang tạo prompt cho record {record_id}")
                    full_content = f"""
                    Mô tả video từ người đăng: {content_text}
                    Transccript của video được lấy từ audio: {transcript if transcript else 'Không có'}
                    Comment của user dưới video: {comments if comments else 'Không có'}
                    """
                    
                    logger.info(f"Đang gọi OpenAI API cho record {record_id}")
                    response = get_completion(full_content, record_id)
                    
                    if response is None:
                        logger.warning(f"Không nhận được response từ OpenAI cho record {record_id}")
                        continue

                    try:
                        logger.info(f"Đang xử lý response cho record {record_id}")
                        ai_message_content = response.choices[0].message.content.strip()
                        logger.info(f"Response content: {ai_message_content[:100]}...")

                        # Loại bỏ markdown code block markers
                        if ai_message_content.startswith('```json'):
                            ai_message_content = ai_message_content[7:]
                        if ai_message_content.endswith('```'):
                            ai_message_content = ai_message_content[:-3]
                        
                        # Trim whitespace
                        ai_message_content = ai_message_content.strip()
                        
                        logger.info(f"Cleaned JSON content: {ai_message_content[:100]}...")
                        
                        # Parse JSON
                        logger.info("Đang parse JSON response")
                        parsed_json = json.loads(ai_message_content)
                        
                        # Chuẩn bị dữ liệu để update
                        logger.info("Đang chuẩn bị dữ liệu để update Airtable")
             
                        extracted_fields = {
                            "AI_comments_persona": parsed_json.get("AI_comments_persona", {}).get("description", "no information"),
                            "AI_comments_painpoints": format_painpoints(parsed_json.get("AI_comments_painpoints", {})),
                            "AI_comments_needs": format_needs(parsed_json.get("AI_comments_needs", [])),
                            "AI_comments_sensitive": format_reverse_to_bullets(parsed_json.get("AI_comments_sensitive", {})),
                            "AI_comments_hidden_insights": get_hidden_insights(parsed_json.get("AI_comments_hidden_insights", "no information"))
                        }
                        
                        # Log extracted fields trước khi update
                        logger.info(f"Extracted fields: {json.dumps(extracted_fields, ensure_ascii=False)}")
                        
                        # Log để kiểm tra giá trị trước khi update
                        logger.info(f"Hidden insights before update: {extracted_fields['AI_comments_hidden_insights']}")
                        
                        # Update Airtable
                        logger.info(f"Đang update record {record_id} vào Airtable")
                        update_airtable_record(base_id, table_name, airtable_api_key, record_id, extracted_fields)
                        logger.info(f"Đã update thành công record {record_id}")
                        
                    except json.JSONDecodeError as e:
                        logger.error(f"Lỗi parse JSON cho record {record_id}: {e}")
                        logger.error(f"JSON content causing error: {ai_message_content}")
                    except Exception as e:
                        logger.error(f"Lỗi xử lý response cho record {record_id}: {e}")
                else:
                    logger.warning(f"Thiếu content_text cho record {record_id}")

            offset = data.get("offset")
            if not offset:
                logger.info("Đã xử lý hết tất cả records")
                break

    except Exception as e:
        logger.error(f"Lỗi trong quá trình chạy: {str(e)}")
        raise e

if __name__ == "__main__":
    if test_openai_connection():
        main()
    else:
        logger.error("Không thể kết nối với Azure OpenAI, dừng chương trình")



2024-10-30 11:54:07,643 - INFO - Kiểm tra kết nối Azure OpenAI
2024-10-30 11:54:08,900 - INFO - HTTP Request: POST https://stepup.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2024-10-30 11:54:08,902 - INFO - Kết nối Azure OpenAI thành công
2024-10-30 11:54:08,902 - INFO - Bắt đầu chạy chương trình
2024-10-30 11:54:08,903 - INFO - Đang fetch dữ liệu từ Airtable...
2024-10-30 11:54:08,903 - INFO - Fetching data from Airtable with params: {'view': '0_extract_2'}
2024-10-30 11:54:11,491 - INFO - Successfully fetched 96 records
2024-10-30 11:54:11,493 - INFO - Số lượng records lấy được: 96
2024-10-30 11:54:11,493 - INFO - Đang xử lý record ID: rec1CCqW8mldEHQDP
2024-10-30 11:54:11,494 - INFO - Dữ liệu của record rec1CCqW8mldEHQDP:
2024-10-30 11:54:11,494 - INFO - Title: ...
2024-10-30 11:54:11,495 - INFO - Content: Games to learn English  #learnenglish #websites #e...
2024-10-30 11:54:11,496 - INFO - Script: [0:00:00]  Nếu

2024-10-30 11:54:22,374 - INFO - Đang update record recOPtWo8WLYmkbcI vào Airtable
2024-10-30 11:54:24,056 - INFO - Successfully updated record recOPtWo8WLYmkbcI
2024-10-30 11:54:24,058 - INFO - Đã update thành công record recOPtWo8WLYmkbcI
2024-10-30 11:54:24,059 - INFO - Đang xử lý record ID: recfsoK2Of5Rcl4az
2024-10-30 11:54:24,060 - INFO - Dữ liệu của record recfsoK2Of5Rcl4az:
2024-10-30 11:54:24,061 - INFO - Title: ...
2024-10-30 11:54:24,061 - INFO - Content: Hope everything will be fine 🙏🏻  #english #fyp ...
2024-10-30 11:54:24,062 - INFO - Script: [0:00:00]  Hãy subscribe cho kênh Ghiền Mì Gõ Để k...
2024-10-30 11:54:24,062 - INFO - Comments: [thorcute1202]
[Cho e xin tên web ạ]

[chauanh738]...
2024-10-30 11:54:24,063 - INFO - Đang tạo prompt cho record recfsoK2Of5Rcl4az
2024-10-30 11:54:24,064 - INFO - Đang gọi OpenAI API cho record recfsoK2Of5Rcl4az
2024-10-30 11:54:24,064 - INFO - Sending request to OpenAI for record recfsoK2Of5Rcl4az
2024-10-30 11:54:27,555 - INFO - HTTP 

2024-10-30 11:54:35,025 - INFO - Hidden insights before update: Người dùng có thể chưa nhận ra rằng việc học từ vựng và ngữ pháp chuyên sâu có thể giúp họ trong các kỳ thi như IELTS, mặc dù họ đang tập trung vào chương trình chuyên Anh.
2024-10-30 11:54:35,026 - INFO - Đang update record recqMYyJ3ZKgmkw69 vào Airtable
2024-10-30 11:54:36,559 - INFO - Successfully updated record recqMYyJ3ZKgmkw69
2024-10-30 11:54:36,562 - INFO - Đã update thành công record recqMYyJ3ZKgmkw69
2024-10-30 11:54:36,563 - INFO - Đang xử lý record ID: recl3Bm10vRdXEPil
2024-10-30 11:54:36,564 - INFO - Dữ liệu của record recl3Bm10vRdXEPil:
2024-10-30 11:54:36,565 - INFO - Title: ...
2024-10-30 11:54:36,566 - INFO - Content: Practice your Englishhhhhhh  #english #learningeng...
2024-10-30 11:54:36,567 - INFO - Script: [0:00:00]  Hãy subscribe cho kênh Ghiền Mì Gõ Để k...
2024-10-30 11:54:36,568 - INFO - Comments: [englishwithjannie_]
[Replika cho b nào cần ạ 🫶🏻]...
2024-10-30 11:54:36,568 - INFO - Đang tạo promp

2024-10-30 11:54:52,087 - INFO - Hidden insights before update: ['Người dùng có thể không nhận ra rằng việc sai sót là một phần tự nhiên của quá trình học, và việc có phản hồi từ người khác có thể giúp họ cải thiện nhanh chóng.', 'Có thể có nhu cầu tìm kiếm các nguồn tài liệu hoặc khóa học bổ sung để hỗ trợ việc học IPA và phát âm.']
2024-10-30 11:54:52,088 - INFO - Đang update record rechx3OYJycFHn6zL vào Airtable
2024-10-30 11:54:53,581 - INFO - Successfully updated record rechx3OYJycFHn6zL
2024-10-30 11:54:53,583 - INFO - Đã update thành công record rechx3OYJycFHn6zL
2024-10-30 11:54:53,583 - INFO - Đang xử lý record ID: rec4KUlFIVakV13fr
2024-10-30 11:54:53,584 - INFO - Dữ liệu của record rec4KUlFIVakV13fr:
2024-10-30 11:54:53,585 - INFO - Title: ...
2024-10-30 11:54:53,586 - INFO - Content: 4 things English beginners should bear in mind che...
2024-10-30 11:54:53,586 - INFO - Script: [0:00:00]  4 lời khuyên dành cho các bạn mới bắt đ...
2024-10-30 11:54:53,587 - INFO - Comments: [

2024-10-30 11:55:09,179 - INFO - Received response from OpenAI for record rec9fEQhBoNch2eAa
2024-10-30 11:55:09,180 - INFO - Đang xử lý response cho record rec9fEQhBoNch2eAa
2024-10-30 11:55:09,181 - INFO - Response content: ```json
{
  "AI_comments_persona": {
    "user_segments": [
      {
        "segment": "Học sinh/sin...
2024-10-30 11:55:09,182 - INFO - Cleaned JSON content: {
  "AI_comments_persona": {
    "user_segments": [
      {
        "segment": "Học sinh/sinh viên",...
2024-10-30 11:55:09,183 - INFO - Đang parse JSON response
2024-10-30 11:55:09,184 - INFO - Đang chuẩn bị dữ liệu để update Airtable
2024-10-30 11:55:09,185 - INFO - Extracted fields: {"AI_comments_persona": "no information", "AI_comments_painpoints": "Thiếu động lực để học tiếng Anh một cách hiệu quả., Cảm thấy áp lực khi phải đạt điểm cao trong các kỳ thi., Có thể gặp khó khăn về tài chính khi tìm kiếm sự trợ giúp học tập., Khó khăn trong việc hiểu và áp dụng ngữ pháp, từ vựng và cấu trúc câu., Có thể đã t

...

2024-10-30 12:02:12,104 - INFO - Hidden insights before update: Người dùng có thể không nhận ra rằng việc học tiếng Anh không chỉ là về ngữ pháp và từ vựng, mà còn liên quan đến việc phát triển kỹ năng giao tiếp và tự tin trong việc sử dụng ngôn ngữ.
2024-10-30 12:02:12,105 - INFO - Đang update record rec90guRtA2upLXnO vào Airtable
2024-10-30 12:02:13,530 - INFO - Successfully updated record rec90guRtA2upLXnO
2024-10-30 12:02:13,531 - INFO - Đã update thành công record rec90guRtA2upLXnO
2024-10-30 12:02:13,533 - INFO - Đang xử lý record ID: recLLMC5mOa7PWkRS
2024-10-30 12:02:13,534 - INFO - Dữ liệu của record recLLMC5mOa7PWkRS:
2024-10-30 12:02:13,535 - INFO - Title: ...
2024-10-30 12:02:13,537 - INFO - Content: Món quà đặc biệt từ VTV#têũ #xuhuongtiktok ...
2024-10-30 12:02:13,539 - INFO - Script: [0:00:00]  Ngày hôm qua, tôi về trễ quá trễ, nên t...
2024-10-30 12:02:13,540 - INFO - Comments: [tnjb76]
[xin vía giỏi english]

[_camm.changg_]
[...
2024-10-30 12:02:13,540 - INFO - Đang t

2024-10-30 12:02:31,210 - INFO - Đang xử lý record ID: recvcDtLCBpeHMCJE
2024-10-30 12:02:31,211 - INFO - Dữ liệu của record recvcDtLCBpeHMCJE:
2024-10-30 12:02:31,211 - INFO - Title: ...
2024-10-30 12:02:31,212 - INFO - Content: Thuyết trình Múa Rối Nước #têũ#thuyettrinh#Muarốin...
2024-10-30 12:02:31,213 - INFO - Script: [0:00:00]  Ở Hải Phòng, tôi rất vui về có một văn ...
2024-10-30 12:02:31,213 - INFO - Comments: [lieunguyetcongtu16]
[Ô Tễu cũng ng HP ạ ! Ngưỡng ...
2024-10-30 12:02:31,214 - INFO - Đang tạo prompt cho record recvcDtLCBpeHMCJE
2024-10-30 12:02:31,215 - INFO - Đang gọi OpenAI API cho record recvcDtLCBpeHMCJE
2024-10-30 12:02:31,216 - INFO - Sending request to OpenAI for record recvcDtLCBpeHMCJE
2024-10-30 12:02:35,851 - INFO - HTTP Request: POST https://stepup.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2024-10-30 12:02:35,857 - INFO - Received response from OpenAI for record recvcDtLCBpeHMCJE
202

2024-10-30 12:02:47,670 - INFO - Hidden insights before update: Nhiều người dùng có thể không nhận ra rằng việc học tiếng Anh không chỉ là về từ vựng mà còn là về sự tự tin và thực hành thường xuyên.
2024-10-30 12:02:47,670 - INFO - Đang update record recFwneZRedWJJ2mH vào Airtable
2024-10-30 12:02:49,166 - INFO - Successfully updated record recFwneZRedWJJ2mH
2024-10-30 12:02:49,168 - INFO - Đã update thành công record recFwneZRedWJJ2mH
2024-10-30 12:02:49,169 - INFO - Đang xử lý record ID: rec3v5T1BheS4a5VP
2024-10-30 12:02:49,171 - INFO - Dữ liệu của record rec3v5T1BheS4a5VP:
2024-10-30 12:02:49,171 - INFO - Title: ...
2024-10-30 12:02:49,172 - INFO - Content: Nghe tin tức trên Vox #learningenglish #têũ ...
2024-10-30 12:02:49,173 - INFO - Script: [0:00:00]  tạo ra một sản phẩm mới hoặc một cách k...
2024-10-30 12:02:49,174 - INFO - Comments: [minhhuong.11]
[Cho c hỏi e xem kênh nào v? C bắt ...
2024-10-30 12:02:49,175 - INFO - Đang tạo prompt cho record rec3v5T1BheS4a5VP
2024-10-30 

2024-10-30 12:03:13,546 - INFO - Đang parse JSON response
2024-10-30 12:03:13,546 - INFO - Đang chuẩn bị dữ liệu để update Airtable
2024-10-30 12:03:13,547 - INFO - Extracted fields: {"AI_comments_persona": "no information", "AI_comments_painpoints": "Cần học cách tỏ tình bằng tiếng Anh", "AI_comments_needs": "- Cần tìm cách học tiếng Anh một cách thú vị và dễ nhớ.\n- Muốn kết hợp việc học tiếng Anh với sở thích âm nhạc.", "AI_comments_sensitive": "- general_reaction: Tích cực\n- explanation: Người dùng thể hiện sự thích thú và hào hứng với nội dung video, cho thấy rằng họ cảm thấy video này hữu ích và thú vị cho việc học tiếng Anh.", "AI_comments_hidden_insights": "Người dùng có thể không nhận ra rằng việc học tiếng Anh qua âm nhạc có thể giúp họ cải thiện khả năng nghe và phát âm một cách tự nhiên."}
2024-10-30 12:03:13,548 - INFO - Hidden insights before update: Người dùng có thể không nhận ra rằng việc học tiếng Anh qua âm nhạc có thể giúp họ cải thiện khả năng nghe và phát âm một 

2024-10-30 12:03:24,699 - INFO - Đang xử lý record ID: rectGNet8Qjjudv5T
2024-10-30 12:03:24,700 - INFO - Dữ liệu của record rectGNet8Qjjudv5T:
2024-10-30 12:03:24,701 - INFO - Title: ...
2024-10-30 12:03:24,701 - INFO - Content: Một tâm hồn được chữa lành và một cái đầu có thêm ...
2024-10-30 12:03:24,702 - INFO - Script: [0:00:00]  từng ngày anh đi, từng ngày anh bước ra...
2024-10-30 12:03:24,703 - INFO - Comments: [migmig737475]
[Chính cái đoạn đó đã an ủi tinh th...
2024-10-30 12:03:24,704 - INFO - Đang tạo prompt cho record rectGNet8Qjjudv5T
2024-10-30 12:03:24,705 - INFO - Đang gọi OpenAI API cho record rectGNet8Qjjudv5T
2024-10-30 12:03:24,705 - INFO - Sending request to OpenAI for record rectGNet8Qjjudv5T
2024-10-30 12:03:28,226 - INFO - HTTP Request: POST https://stepup.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2024-10-30 12:03:28,232 - INFO - Received response from OpenAI for record rectGNet8Qjjudv5T
202